<a href="https://www.kaggle.com/code/jordantasker/e-commerce-project?scriptVersionId=139453849" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1 style="font-size: 28px; font-family: Helvetica; color: #000; font-weight: bold; text-align: center;">Introduction</h1> <p style="font-size: 18px; font-family: Arial; line-height: 1.5; color: #222;">
<b>Simple exploratory data analysis on a transactional dataset containing all transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based online retailer selling unique all-occasion gifts. The company's customers are primarily wholesalers.</b>

<h2 style="font-size: 28px; font-family: Helvetica; color: #000; font-weight: bold; text-align: left;">1.0 Basic EDA</h2> 
<p 
   style="font-size: 18px; font-family: Arial; line-height: 1.5; color: #222; text-align: left;">
<b>Gaining a better understanding of the dataset</b>

</p>

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns

import warnings
# Ignores warnings from Seaborn
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

import missingno as msno # missing data visualization module for Python
import pandas_profiling

import gc
import datetime

%matplotlib inline
color = sns.color_palette()

from plotly.offline import init_notebook_mode, iplot

print("packages loaded successfully")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


packages loaded successfully


In [2]:
#Import data source file for EDA and print message to show successful/unsuccessful file import

try:
   df = pd.read_csv('/kaggle/input/ecommerce-data/data.csv', encoding = 'latin')
except:
    print("Error occurred while importing the data")
else:
    print("data imported successfully")

data imported successfully


In [3]:
#Print first few rows for overview of dataset

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
#Make column headings easier to work with and to read.

try:
 df.rename(index=str, columns={'InvoiceNo': 'invoice_no',
                              'StockCode' : 'stock_code',
                              'Description' : 'description',
                              'Quantity' : 'quantity',
                              'InvoiceDate' : 'invoice_date',
                              'UnitPrice' : 'unit_price',
                              'CustomerID' : 'cust_id',
                              'Country' : 'country'}, inplace=True)

#Using try, except, else to handle any possible error messages.
except:
    print("Error")
    
df.head()


,invoice_no,stock_code,description,quantity,invoice_date,unit_price,cust_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


<h3 style="font-size: 28px; font-family: Helvetica; color: #000; font-weight: bold; text-align: left;">2.0 In-Depth EDA</h3> 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice_no    541909 non-null  object 
 1   stock_code    541909 non-null  object 
 2   description   540455 non-null  object 
 3   quantity      541909 non-null  int64  
 4   invoice_date  541909 non-null  object 
 5   unit_price    541909 non-null  float64
 6   cust_id       406829 non-null  float64
 7   country       541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 37.2+ MB


In [6]:
df.describe()

,quantity,unit_price,cust_id
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


<p style="font-size: 16px; font-family: Arial; line-height: 1.5; color: #222;">
<b>From looking at the above, we can see that we are missing some customer IDs, have negative values under quantity and unit price and also some insights, such as: 50-75% of transactions contain 3-10 items, with most items priced between £2 and £5.  </b>

In [7]:
#Find NULL values in dataset

df.isnull().sum().sort_values(ascending=False)

cust_id         135080
description       1454
invoice_no           0
stock_code           0
quantity             0
invoice_date         0
unit_price           0
country              0
dtype: int64

<p style="font-size: 16px; font-family: Arial; line-height: 1.5; color: #222;">
<b>We also have a significant amount of NULL values within the dataset that would usually require further investigation to understand how best to deal with the missing values.</b>
    <p style="font-size: 16px; font-family: Arial; line-height: 1.5; color: #222;">
        <b> NULL values are usually resolved by either:
<p style="font-size: 16px; font-family: Arial; line-height: 1.5; color: #222;">
* <b>deleting the rows with null values<b>
    <p style="font-size: 16px; font-family: Arial; line-height: 1.5; color: #222;">
* <b>deleting the entire column with null values<b>
        <p style="font-size: 16px; font-family: Arial; line-height: 1.5; color: #222;">
* <b>imputing the data with the mean or median values or keep the cells as NULL if they add context to the dataset<b>             

In [8]:
#Check datatypes for each column:

df.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
cust_id         float64
country          object
dtype: object

<h4 style="font-size: 28px; font-family: Helvetica; color: #000; font-weight: bold; text-align: left;">3.0 Data Cleaning</h4>

<p style="font-size: 16px; font-family: Arial; line-height: 1.5; color: #222;">
<b>The invoice date is currently set as an object so we will convert this to datetime as it is easier to work with.  </b>

In [9]:
#Cast invoice_date to datetime

df['invoice_date'] = pd.to_datetime(df['invoice_date'])

In [10]:
#Check above was done correctly

df.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
cust_id                float64
country                 object
dtype: object

In [11]:
#In this case, we will drop NULL values for simplicity

try:
 df_new = df.dropna()
except:
    print("Error")
else:
    print("Dropped NAs")

Dropped NAs


<p style="font-size: 16px; font-family: Arial; line-height: 1.5; color: #222;">
<b>Removing negative values  </b>

In [12]:
#Round values to 2 decimal places

df_new.describe().round(2)

,quantity,unit_price,cust_id
count,406829.00,406829.00,406829.00
mean,12.06,3.46,15287.69
std,248.69,69.32,1713.60
min,-80995.00,0.00,12346.00
25%,2.00,1.25,13953.00
50%,5.00,1.95,15152.00
75%,12.00,3.75,16791.00
max,80995.00,38970.00,18287.00


In [13]:
#Remove negative quantities

try:
 df_new = df_new[df_new.quantity > 0] #removing quantity which is in negative.
except:
    print("Error")
else:
    print("Removed Negative Quantities")

Removed Negative Quantities


In [14]:
#Change customer ID to string datatype

try:
 df_new['cust_id'] = df_new['cust_id'].astype('str')
except:
    print("Error")
else:
    print("changed datatype")

changed datatype


In [15]:
#Add total_sales column

df_new['total_sales'] = df_new['quantity'] * df_new['unit_price']

try:
 df_new = df_new[['invoice_no','invoice_date','stock_code','description',
                 'quantity','unit_price','total_sales','cust_id','country']]
except:
    print("Error")
else:
    print("Column Added")

Column Added


In [16]:
#Quick overview of current dataset

df_new.head()

,invoice_no,invoice_date,stock_code,description,quantity,unit_price,total_sales,cust_id,country
0,536365,2010-12-01 08:26:00,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850.0,United Kingdom
1,536365,2010-12-01 08:26:00,71053,WHITE METAL LANTERN,6,3.39,20.34,17850.0,United Kingdom
2,536365,2010-12-01 08:26:00,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,22.00,17850.0,United Kingdom
3,536365,2010-12-01 08:26:00,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,20.34,17850.0,United Kingdom
4,536365,2010-12-01 08:26:00,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,20.34,17850.0,United Kingdom


In [17]:
#Add Month, Day and Hour columns to the dataset

df_new.insert(loc=2, column='year_month', value=df_new['invoice_date'].map(lambda x: 100*x.year + x.month))
df_new.insert(loc=3, column='month', value=df_new.invoice_date.dt.month)
# +1 to make Monday=1.....until Sunday=7
df_new.insert(loc=4, column='day', value=(df_new.invoice_date.dt.dayofweek)+1)
df_new.insert(loc=5, column='hour', value=df_new.invoice_date.dt.hour)

df_new.head()

,invoice_no,invoice_date,year_month,month,day,hour,stock_code,description,quantity,unit_price,total_sales,cust_id,country
0,536365,2010-12-01 08:26:00,201012,12,3,8,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850.0,United Kingdom
1,536365,2010-12-01 08:26:00,201012,12,3,8,71053,WHITE METAL LANTERN,6,3.39,20.34,17850.0,United Kingdom
2,536365,2010-12-01 08:26:00,201012,12,3,8,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,22.00,17850.0,United Kingdom
3,536365,2010-12-01 08:26:00,201012,12,3,8,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,20.34,17850.0,United Kingdom
4,536365,2010-12-01 08:26:00,201012,12,3,8,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,20.34,17850.0,United Kingdom


In [18]:
#Create line chart to show Total Number of Order By Month

import plotly.graph_objects as go

init_notebook_mode(connected=True)

# Convert invoice_date to datetime format
df_new['invoice_date'] = pd.to_datetime(df_new['invoice_date'])

# Create a new column with year and month
df_new['year_month'] = df_new['invoice_date'].dt.to_period('M')

# Group by year_month and count the number of unique invoices
orders_by_month = df_new.groupby('year_month')['invoice_no'].nunique().reset_index()

# Line chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=orders_by_month['year_month'].apply(lambda x: x.strftime('%Y-%m')), y=orders_by_month['invoice_no'], mode='lines'))
fig.update_layout(title='Total Number of Orders by Month', xaxis_title='Month', yaxis_title='Number of Orders')
fig.show()

<p style="font-size: 16px; font-family: Arial; line-height: 1.5; color: #222;">
<b>The above shows the pre-christmas period as the period with the most online orders, which coincides with Black Friday deals and early Christmas shopping.  </b>

In [19]:
#Create bar chart to show orders by day

weekday_map = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}

orders_by_day = df_new.groupby('day').agg({'invoice_no': 'nunique'}).reset_index()
orders_by_day['day'] = orders_by_day['day'].map(weekday_map)

# Create bar chart trace
trace = go.Bar(x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], y=[20, 30, 25, 40, 15, 10, 5])

# Create layout
layout = go.Layout(title='Number of Orders by Day', xaxis_title='Day of Week', yaxis_title='Number of Orders')

# Create figure with the trace and layout
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

<p style="font-size: 16px; font-family: Arial; line-height: 1.5; color: #222;">
<b>The bar chart above shows that Thursday tends to be the busiest day for online orders  </b>

In [20]:
#Create bar chart to show number of orders by hour in day

orders_by_hour = df_new.groupby('hour')['invoice_no'].count().reset_index(name='count')
fig = go.Figure(go.Bar(
            x=orders_by_hour['hour'],
            y=orders_by_hour['count'],
            ))
fig.update_layout(title='Total Number of Orders by Hour', xaxis_title='Hour', yaxis_title='Number of Orders')
fig.show()


In [21]:
#Create total number of orders by country chart

# Group data by country and count number of unique invoice_no for each country
orders_by_country = df_new.groupby('country')['invoice_no'].nunique().reset_index(name='total_orders_country')
orders_by_country = orders_by_country.sort_values('total_orders_country', ascending=False)

# Create bar chart to visualize the number of orders for each country
fig = go.Figure(go.Bar(
            x=orders_by_country['country'],
            y=orders_by_country['total_orders_country'],
            ))
fig.update_layout(title='Total Number of Orders by Country', xaxis_title='Countries', yaxis_title='Number of Orders')
fig.show()

In [22]:
#Create 2nd bar chart for total number of orders by country minus UK for better insight to foreign online sales

orders_by_country = orders_by_country[orders_by_country['country'] != 'United Kingdom']
orders_by_country = orders_by_country.sort_values('total_orders_country', ascending=False)

# Create bar chart to visualize the number of orders for each country
fig = go.Figure(go.Bar(
            x=orders_by_country['country'],
            y=orders_by_country['total_orders_country'],
            ))
fig.update_layout(title='Total Number of Orders by Other Countries (Excl UK)', xaxis_title='Countries', yaxis_title='Number of Orders')
fig.show()

In [23]:
#Create bar chart to show total sales by country (£ GBP)

total_sales_by_country = df_new.groupby('country')['total_sales'].sum().reset_index()
total_sales_by_country = total_sales_by_country.sort_values('total_sales', ascending=False)

fig = go.Figure(go.Bar(
            x=total_sales_by_country['country'],
            y=total_sales_by_country['total_sales'],
            ))
fig.update_layout(title='Total Sales by Country (£ GBP)', xaxis_title='Countries', yaxis_title='Total Sales (£ GBP)')
fig.show()

In [24]:
#Create 2nd bar chart for total sales by country minus UK for better insight to total foreign online sales

total_sales_by_country = df_new.groupby('country')['total_sales'].sum().reset_index()
total_sales_by_country = total_sales_by_country.sort_values('total_sales', ascending=False)
total_sales_by_country = total_sales_by_country[total_sales_by_country['country'] != 'United Kingdom']

fig = go.Figure(go.Bar(
            x=total_sales_by_country['country'],
            y=total_sales_by_country['total_sales'],
            ))
fig.update_layout(title='Total Sales by other Countries (£ GBP)', xaxis_title='Countries', yaxis_title='Total Sales (£ GBP)')
fig.show()